In [ ]:
import torch

: 

In [2]:
import pandas as pd
data=pd.read_csv("ner_dataset.csv",encoding="latin1")

In [3]:
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
data=data.fillna(method="ffill")

In [5]:
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
data["Sentence #"]= LabelEncoder().fit_transform(data["Sentence #"])

In [8]:
data.head()

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [9]:
data.rename(columns={"Sentence #": "sentence_id","Word":"words","Tag":"labels",},inplace=True)

In [10]:
data["labels"]=data["labels"].str.upper()
data.head(5)

,sentence_id,words,POS,labels
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [11]:
X=data[["sentence_id","words"]]
Y=data["labels"]

In [12]:
x_train,x_test,y_train,y_test =train_test_split(X,Y,test_size=0.2)

In [13]:
# building up train data and test data
train_data=pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data=pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [14]:
train_data

,sentence_id,words,labels
618237,20297,consulting,O
17152,45449,British,B-GPE
561850,17436,attack,O
902416,34684,President,B-PER
817431,30405,nomination,O
...,...,...,...
363607,7374,it,O
336556,6004,group,O
556581,17180,said,O
53399,15768,Sabbath,I-PER


Model Training

In [15]:
from simpletransformers.ner import NERModel,NERArgs

In [16]:
label=data["labels"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [17]:
args=NERArgs()
args.num_train_epochs=1
args.learning_rate=1e-4
args.overwrite_output_dir=True
args.train_batch_size=32
args.eval_batch_size=32

In [18]:
model= NERModel('bert','bert-base-cased',labels=label,args=args,use_cuda=False) 

C:\Users\visha\anaconda3\New_folder\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\visha\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.tran

In [ ]:
model.train_model(train_data,eval_data=test_data,acc=accuracy_score)

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

In [ ]:
result,model_outputs,preds_list=model.eval_model(test_data)

In [ ]:
result

In [ ]:
prediction,model_output=model.predict(["what is the new name of bangalore"])

In [ ]:
prediction